In [16]:
# stdlib
import enum
import struct
import time

# util
from see import see

# rpi hardware
from RPi import GPIO as gpio
import smbus

# Refs

## SMBus/I<sup>2</sup>C

* https://www.kernel.org/doc/Documentation/i2c/smbus-protocol (`smbus` seems to wrap these fairly directly)
* http://raspberry-projects.com/pi/programming-in-python/i2c-programming-in-python/using-the-i2c-interface-2

## MCP9808

* https://cdn-shop.adafruit.com/datasheets/MCP9808.pdf

In [2]:
heater = 5
other = 6

gpio.setmode(gpio.BCM)  # corresponds to the pins listed on the perma-proto hat
gpio.setup(heater, gpio.OUT, initial=gpio.LOW)
gpio.output(heater, gpio.HIGH)  # heater ON
time.sleep(0.5)
gpio.output(heater, gpio.LOW)  # heater OFF
gpio.cleanup()  # reset pin state

In [2]:
bus = smbus.SMBus(1)

In [191]:
def byte_flip(word):
    return int.from_bytes((word).to_bytes(2, "little"), "big")


def byte_flip2(word):
    msb, lsb = divmod(word, 256)
    return lsb * 256 + msb


for n in range(2**15):
    assert byte_flip(n) == byte_flip2(n)
    

class MCP9808:
    class Registers(enum.IntEnum):
        CONFIG = 0b0001
        T_UPPER = 0b0010
        T_LOWER = 0b0011
        T_CRIT = 0b0100
        TEMP = 0b0101
        MFC_ID = 0b0110
        DEV_ID = 0b0111
        RESOLUTION = 0b1000
        
    class Resolution(enum.IntEnum):
        C_2 = 0b00  # t_CONV = 30 ms typ
        C_4 = 0b01  # t_CONV = 60 ms typ
        C_8 = 0b10  # t_CONV = 130 ms typ
        C_16 = 0b11  # t_CONV = 250 ms typ, DEFAULT
        
    class Flags(enum.IntFlag):
        T_CRITICAL = 0x8000
        T_HIGH = 0x4000
        T_LOW = 0x2000
        
    BASE_ADDRESS = 0x18
    
    # second byte is LSB on the datasheet, but smbus reads it as MSB. these values
    # are byte-flipped to match what smbus will give.
    MFC_ID = 0b0000_0000_0101_0100
    DEV_ID = 0b0000_0100_0000_0000
        
    def __init__(self, smbus, address_bits=0b000):
        self.smbus = smbus
        self.address = self.BASE_ADDRESS + address_bits
        
        self._validate()
        
    def _read_word(self, register):
        word = self.smbus.read_word_data(self.address, register)
        return byte_flip(word)
        
    def _validate(self):
        mfc_word = self._read_word(self.Registers.MFC_ID)
        if mfc_word != self.MFC_ID:
            raise RuntimeError(f"manufacturer ID does not match! (received {mfc_word})")
        
        dev_word = self._read_word(self.Registers.DEV_ID)
        if dev_word != self.DEV_ID:
            raise RuntimeError(f"device ID/revision does not match! (received {dev_word})")
        
    def _read_temp(self):
        word = self._read_word(self.Registers.TEMP)
        
        # ripped as directly as possible from example 5.1 
        # in DS25095A page 25
        
        ## THIS IS BROKEN FOR NEGATIVE TEMPS

        UpperByte, LowerByte = divmod(word, 2**8)

        # if ((UpperByte & 0x80) == 0x80){ //TA ³ TCRIT
        # }
        T_CRIT = (UpperByte & 0x80) == 0x80
        # if ((UpperByte & 0x40) == 0x40){ //TA > TUPPER
        # }
        T_HIGH = (UpperByte & 0x40) == 0x40
        # if ((UpperByte & 0x20) == 0x20){ //TA < TLOWER
        # }
        T_LOW = (UpperByte & 0x20) == 0x20

        UpperByte = UpperByte & 0x1F #//Clear flag bits
        # if ((UpperByte & 0x10) == 0x10){ //TA < 0°C
        if ((UpperByte & 0x10) == 0x10):
            # UpperByte = UpperByte & 0x0F; //Clear SIGN
            UpperByte = UpperByte & 0x0F#; //Clear SIGN
            # Temperature = 256 - (UpperByte x 16 + LowerByte / 16);
            Temperature = 256 - (UpperByte * 16 + LowerByte / 16)
        # }else //TA ³ 0°C
        else:
            Temperature = (UpperByte * 16 + LowerByte / 16)
        # //Temperature = Ambient Temperature (°C)

        return T_CRIT, T_HIGH, T_LOW, Temperature
        
    def _read_temp_2(self):
        word = self._read_word(self.Registers.TEMP)
        flags = self.Flags(word & 0xE000)
        
        if word & 0x1000:
            word = word & 0x0FFF
            return flags, (word / 16) - 256

        return flags, (word & 0x1FFF) / 16
        
    @property
    def T(self):
        flags, temp = self._read_temp_2()
        return temp
    
    @property
    def resolution(self):
        res = self.smbus.read_byte_data(self.address, self.Registers.RESOLUTION)
        return self.Resolution(res)
    
    @resolution.setter
    def resolution(self, value):
        self.smbus.write_byte_data(self.address, self.Registers.RESOLUTION, value)

In [114]:
def ref_function(word):
    # ripped as directly as possible from example 5.1 
    # in DS25095A page 25

    UpperByte, LowerByte = divmod(word, 2**8)

    # if ((UpperByte & 0x80) == 0x80){ //TA ³ TCRIT
    # }
    T_CRIT = (UpperByte & 0x80) == 0x80
    # if ((UpperByte & 0x40) == 0x40){ //TA > TUPPER
    # }
    T_HIGH = (UpperByte & 0x40) == 0x40
    # if ((UpperByte & 0x20) == 0x20){ //TA < TLOWER
    # }
    T_LOW = (UpperByte & 0x20) == 0x20

    UpperByte = UpperByte & 0x1F #//Clear flag bits
    # if ((UpperByte & 0x10) == 0x10){ //TA < 0°C
    if ((UpperByte & 0x10) == 0x10):
        # UpperByte = UpperByte & 0x0F; //Clear SIGN
        UpperByte = UpperByte & 0x0F#; //Clear SIGN
        # Temperature = 256 - (UpperByte x 16 + LowerByte / 16);
        Temperature = 256 - (UpperByte * 16 + LowerByte / 16)
    # }else //TA ³ 0°C
    else:
        Temperature = (UpperByte * 16 + LowerByte / 16)
    # //Temperature = Ambient Temperature (°C)

#     return T_CRIT, T_HIGH, T_LOW, Temperature
    return Temperature

In [115]:
def adafruit_func(word):
    buf = bytearray(3)
    buf[1], buf[2] = divmod(word, 256)
    
    buf[1] = buf[1] & 0x1f
    if buf[1] & 0x10 == 0x10:
        buf[1] = buf[1] & 0x0f
        return (buf[1] * 16 + buf[2] / 16.0) - 256
    return buf[1] * 16 + buf[2] / 16.0

In [122]:
def pract_func(word):
    msb, lsb = divmod(word, 256)
    
    if msb & 0x10:
        msb = msb & 0x0f
        return (msb * 16 + lsb / 16) - 256
    
    msb = msb & 0x1f
    return msb * 16 + lsb / 16

In [124]:
def pract_func2(word):
    if word & 0x1000:
        word = word & 0x0FFF
        return (word / 16) - 256
    
    return (word & 0x1FFF) / 16

In [117]:
adafruit_func(0b1_1111_1111_1111)

-0.0625

In [116]:
ref_function(0b1_1111_1111_1111)

0.0625

In [126]:
match = 0
for n in range(2**16):
    if adafruit_func(n) == pract_func2(n):
        match += 1
match

65536

In [48]:
smbus_device = 1
bus = smbus.SMBus(smbus_device)

In [192]:
mcp9808 = MCP9808(bus, 0b000)

In [182]:
mcp9808.resolution = mcp9808.Resolution.C_16

In [98]:
see(bus)

    <                        <=                       ==
    !=                       >                        >=
    dir()                    hash()                   help()
    repr()                   str()                    .block_process_call()
    .close()                 .dealloc()               .open()
    .pec                     .process_call()          .read_block_data()
    .read_byte()             .read_byte_data()        .read_i2c_block_data()
    .read_word_data()        .write_block_data()      .write_byte()
    .write_byte_data()       .write_i2c_block_data()
    .write_quick()           .write_word_data()

In [10]:
# Resolution settings
HALF_C = 0x0
QUARTER_C = 0x1
EIGHTH_C = 0x2
SIXTEENTH_C = 0x3

class MCP9808:
    """Interface to the MCP9808 temperature sensor."""

    # alert_lower_temperature_bound
    # alert_upper_temperature_bound
    # critical_temperature
    # temperature
    # temperature_resolution

    def __init__(self, smbus, address=0x18):
#         self.i2c_device = I2CDevice(i2c_bus, address)
        self.smbus = smbus

        # Verify the manufacturer and device ids to ensure we are talking to
        # what we expect.
        self.buf = bytearray(3)
        self.buf[0] = 0x06
#         with self.i2c_device as i2c:
#             i2c.write(self.buf, end=1, stop=False)
#             i2c.readinto(self.buf, start=1)
        self.smbus 

        ok = self.buf[2] == 0x54 and self.buf[1] == 0

        # Check device id.
        self.buf[0] = 0x07
        with self.i2c_device as i2c:
            i2c.write(self.buf, end=1, stop=False)
            i2c.readinto(self.buf, start=1)

        if not ok or self.buf[1] != 0x04:
            raise ValueError("Unable to find MCP9808 at i2c address " + str(hex(address)))

    @property
    def temperature(self):
        """Temperature in celsius."""
        self.buf[0] = 0x05
        with self.i2c_device as i2c:
            i2c.write(self.buf, end=1, stop=False)
            i2c.readinto(self.buf, start=1)

        # Clear flags from the value
        self.buf[1] = self.buf[1] & 0x1f
        if self.buf[1] & 0x10 == 0x10:
            self.buf[1] = self.buf[1] & 0x0f
            return (self.buf[1] * 16 + self.buf[2] / 16.0) - 256
        return self.buf[1] * 16 + self.buf[2] / 16.0

In [9]:
mcp9808_address = 0x18


In [ ]:
bus.write_i2c_block_data(address, reg_write_dac, msg)

# Heat Pulse

In [208]:
%matplotlib inline

import matplotlib
matplotlib.use("agg")

import matplotlib.pyplot as plt

In [211]:
f, ax = plt.subplots()
ax.plot([1, 2, 5, 15])

In [212]:
f.show()

In [213]:
bus.close()